In [1]:
!nvidia-smi

Sun Mar 16 19:25:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers==4.45.2
!pip install datasets
!pip install rouge-score
!pip install pymorphy3
!pip install peft

In [3]:
!git clone https://github.com/RefalMachine/llmtf_open
%cd llmtf_open

fatal: destination path 'llmtf_open' already exists and is not an empty directory.
/content/llmtf_open


In [4]:
!git lfs install
!git clone https://huggingface.co/datasets/RussianNLP/russian_super_glue/

Updated git hooks.
Git LFS initialized.
Cloning into 'russian_super_glue'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70 (from 1)
Unpacking objects: 100% (70/70), 31.51 KiB | 1.21 MiB/s, done.
Filtering content: 100% (9/9), 60.74 MiB | 32.89 MiB/s, done.


## Исследуем количество ответов "да" и "нет"

In [16]:
from datasets import load_dataset

dataset = load_dataset("RussianNLP/russian_super_glue", "danetqa")['train']

label_counts = {
    "0": 0,
    "1": 0
}

for sample in dataset:
    label = str(sample['label'])
    label_counts[label] += 1

# Результаты
print(f"Всего примеров: {len(dataset)}")
print(f"Ответов 'Нет': {label_counts['0']}")
print(f"Ответов 'Да': {label_counts['1']}")
print(f"Соотношение: {label_counts['1'] / len(dataset):.1%} ответов 'Да'")

Всего примеров: 1749
Ответов 'Нет': 688
Ответов 'Да': 1061
Соотношение: 60.7% ответов 'Да'


In [47]:
from llmtf.base import SimpleFewShotHFTask
from llmtf.metrics import mean, f1_macro_score
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm
from typing import Dict, List
from datasets import load_dataset
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score

class DaNetQA(SimpleFewShotHFTask):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.method = 'calculate_tokens_proba'
        self._max_new_tokens = 1

    @classmethod
    def name(cls):
        return 'danetqa'

    @property
    def choices(self):
        return ["1", "0"]

    def aggregation(self) -> Dict:
        return {
        "acc": mean,
        "f1_weighted": lambda results: f1_score(
            [y_true for y_true, _ in results],
            [y_pred for _, y_pred in results],
            average="weighted"
        )
    }


    def evaluate(self, sample, y_pred) -> Dict:
        y_true = int(sample['outputs'])
        y_pred = int(max(y_pred.items(), key=lambda x: x[1])[0])
        return {"acc": y_true == y_pred, "f1_weighted": [y_true, y_pred]}

    def dataset_args(self) -> Dict:
        return {'path': 'RussianNLP/russian_super_glue', 'name': 'danetqa'}

    def _convert_dataset(self, dataset):
        dataset = dataset['train']
        outputs = [str(sample['label']) for sample in dataset]
        dataset = dataset.add_column('outputs', outputs)
        return dataset

    def _load_dataset(self, model, max_len, max_sample_per_dataset, few_shot_count) -> List:
        dataset = self._convert_dataset(load_dataset(**self.dataset_args()))

        prompt_dataset = dataset.select(range(min(1000, len(dataset))))
        test_dataset = dataset.select(range(min(1000, len(dataset)), len(dataset)))

        test_dataset = test_dataset.select(range(min(max_sample_per_dataset, len(test_dataset))))
        prompt_start = self.prompt_dataset_start_idx()
        prompt_dataset = prompt_dataset.select(range(prompt_start, min(prompt_start + few_shot_count, len(prompt_dataset))))

        return [{'messages': self._prepare_messages(sample, model, max_len, few_shot_count, prompt_dataset), 'sample': sample} for sample in tqdm(test_dataset)]

    def create_messages(self, sample, with_answer):
        messages = []
        instruction_user = 'Тебе будет задан вопрос. Ответь "1" (да) или "0" (нет). Предоставь ответ в виде одной цифры.\n\nВопрос: {question}'
        instruction_bot = 'Ответ: {outputs}'
        instruction_bot_incomplete = 'Ответ:'

        user_content = instruction_user.format(question=sample['question'])
        bot_content = instruction_bot.format(outputs=sample['outputs']) if with_answer else instruction_bot_incomplete

        messages.append({'role': 'user', 'content': user_content})
        messages.append({'role': 'bot', 'content': bot_content})

        return messages

    def test_split_name(self) -> str:
        return ''

    def prompt_split_name(self) -> str:
        return ''

    def get_answer(self, sample):
        return ' ' + sample['outputs']


In [48]:
task = DaNetQA()

# Модель 1

In [7]:
from llmtf.model import HFModel

model = HFModel(attn_implementation='sdpa', device_map='cuda')
model.from_pretrained('RefalMachine/RuadaptQwen2.5-1.5B-instruct')

model.generation_config.max_new_tokens = 200
model.generation_config.repetition_penalty = 1.0
model.generation_config.do_sample = False
model.generation_config.temperature = 0.0

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/2.28M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

INFO: 2025-03-16 19:29:06,882: llmtf.base.hfmodel: Set eos_token_id in generation_config to [145111]
INFO:llmtf.base.hfmodel:Set eos_token_id in generation_config to [145111]
INFO: 2025-03-16 19:29:06,886: llmtf.base.hfmodel: Model id: RefalMachine/RuadaptQwen2.5-1.5B-instruct
INFO:llmtf.base.hfmodel:Model id: RefalMachine/RuadaptQwen2.5-1.5B-instruct
INFO: 2025-03-16 19:29:06,888: llmtf.base.hfmodel: Leading space: False
INFO:llmtf.base.hfmodel:Leading space: False


In [49]:
from llmtf.evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_dataset(
    task=task,
    model=model,
    output_dir='./DaNetQA',
    max_len=4000,
    few_shot_count=0,
    generation_config=None, # will use model.generation_config by default
    batch_size=4,
    max_sample_per_dataset=200
)

INFO: 2025-03-16 20:37:54,256: llmtf.base.hfmodel: Updated generation_config.eos_token_id: [145111]
INFO:llmtf.base.hfmodel:Updated generation_config.eos_token_id: [145111]
INFO: 2025-03-16 20:37:54,257: llmtf.base.hfmodel: Updated generation_config.stop_strings: ['<|im_end|>']
INFO:llmtf.base.hfmodel:Updated generation_config.stop_strings: ['<|im_end|>']
100%|██████████| 200/200 [00:00<00:00, 2968.29it/s]
INFO: 2025-03-16 20:37:56,090: llmtf.base.danetqa: Loading Dataset: 1.83s
INFO:llmtf.base.danetqa:Loading Dataset: 1.83s
100%|██████████| 50/50 [00:16<00:00,  3.12it/s]
INFO: 2025-03-16 20:38:12,146: llmtf.base.danetqa: Processing Dataset: 16.05s
INFO:llmtf.base.danetqa:Processing Dataset: 16.05s
INFO: 2025-03-16 20:38:12,148: llmtf.base.danetqa: Results for danetqa:
INFO:llmtf.base.danetqa:Results for danetqa:
INFO: 2025-03-16 20:38:12,156: llmtf.base.danetqa: {'acc': 0.57, 'f1_weighted': 0.572088480625066}
INFO:llmtf.base.danetqa:{'acc': 0.57, 'f1_weighted': 0.572088480625066}


In [50]:
!cat ./DaNetQA/danetqa_total.jsonl

{
    "task_name": "danetqa",
    "results": {
        "acc": 0.57,
        "f1_weighted": 0.572088480625066
    },
    "leaderboard_result": 0.571044240312533
}


# Модель 2

In [51]:
model2 = HFModel(attn_implementation='sdpa', device_map='cuda')
model2.from_pretrained('TinyLlama/TinyLlama-1.1B-Chat-v1.0')

model2.generation_config.max_new_tokens = 200
model2.generation_config.repetition_penalty = 1.0
model2.generation_config.do_sample = False
model2.generation_config.temperature = 0.0

INFO: 2025-03-16 20:38:27,743: llmtf.base.hfmodel: _check_if_leading_space: "[29871, 29896]"
INFO:llmtf.base.hfmodel:_check_if_leading_space: "[29871, 29896]"
INFO: 2025-03-16 20:38:27,745: llmtf.base.hfmodel: Set eos_token_id in generation_config to [2]
INFO:llmtf.base.hfmodel:Set eos_token_id in generation_config to [2]
INFO: 2025-03-16 20:38:27,747: llmtf.base.hfmodel: Model id: TinyLlama/TinyLlama-1.1B-Chat-v1.0
INFO:llmtf.base.hfmodel:Model id: TinyLlama/TinyLlama-1.1B-Chat-v1.0
INFO: 2025-03-16 20:38:27,748: llmtf.base.hfmodel: Leading space: True
INFO:llmtf.base.hfmodel:Leading space: True


In [52]:
from llmtf.evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_dataset(
    task=task,
    model=model2,
    output_dir='./DaNetQA2',
    max_len=4000,
    few_shot_count=0,
    generation_config=None, # will use model.generation_config by default
    batch_size=4,
    max_sample_per_dataset=200
)

INFO: 2025-03-16 20:38:30,987: llmtf.base.hfmodel: Updated generation_config.eos_token_id: [2]
INFO:llmtf.base.hfmodel:Updated generation_config.eos_token_id: [2]
INFO: 2025-03-16 20:38:30,988: llmtf.base.hfmodel: Updated generation_config.stop_strings: ['</s>']
INFO:llmtf.base.hfmodel:Updated generation_config.stop_strings: ['</s>']
100%|██████████| 200/200 [00:00<00:00, 2897.45it/s]
INFO: 2025-03-16 20:38:33,047: llmtf.base.danetqa: Loading Dataset: 2.06s
INFO:llmtf.base.danetqa:Loading Dataset: 2.06s
100%|██████████| 50/50 [00:15<00:00,  3.29it/s]
INFO: 2025-03-16 20:38:48,242: llmtf.base.danetqa: Processing Dataset: 15.19s
INFO:llmtf.base.danetqa:Processing Dataset: 15.19s
INFO: 2025-03-16 20:38:48,246: llmtf.base.danetqa: Results for danetqa:
INFO:llmtf.base.danetqa:Results for danetqa:
INFO: 2025-03-16 20:38:48,252: llmtf.base.danetqa: {'acc': 0.62, 'f1_weighted': 0.4783950617283951}
INFO:llmtf.base.danetqa:{'acc': 0.62, 'f1_weighted': 0.4783950617283951}


In [53]:
!cat ./DaNetQA2/danetqa_total.jsonl

{
    "task_name": "danetqa",
    "results": {
        "acc": 0.62,
        "f1_weighted": 0.4783950617283951
    },
    "leaderboard_result": 0.5491975308641975
}
